# Vulnerability indices

In [1]:
import numpy as np
import pandas as pd
from multiprocess import Pool
import time
from Extract_dataV2 import extractData  
import fullmodel_v1_8
import copy
import os


In [8]:
# Constants
MAP_COLOR = "inferno"
SAVE_HEAT = False
SAVE_HEAT_SUB = True
HEAT_MAP_Z_MD_E_SPP2 = True
# TEMP = 0.6
NUM_PROCESSES = 12 # os.cpu_count()

Z_values = list(np.around(np.arange(0, 2.75+0.035, 0.035), 2))
d_values = list(np.around(np.arange(0.02, 0.12+0.0012, 0.0012), 4))

In [9]:
len(Z_values), len(d_values)

(80, 85)

In [10]:
InfoLakes = {'PIGEON LAKE': {'labels': ['MICROCYSTIN, TOTAL',
                              'PHOSPHORUS TOTAL DISSOLVED',
                              'OXYGEN DISSOLVED (FIELD METER)',
                              'Total cyanobacterial cell count (cells/mL)',
                              'TEMPERATURE WATER'],
                              'Years': ['2021'],
                              'DataBase':'merged_water_quality_data.csv',
                              'InitialP':0.05,
                              'InitialO2': 6  },
                
             'PINE LAKE': {'labels': ['MICROCYSTIN, TOTAL',
                              'PHOSPHORUS TOTAL DISSOLVED',
                              'OXYGEN DISSOLVED (FIELD METER)',
                              'Total cyanobacterial cell count (cells/mL)',
                              'TEMPERATURE WATER'],
                              'Years': ['2017'],
                              'DataBase':'merged_water_quality_data.csv',
                              'InitialP':0.4,
                              'InitialO2': 6  },
                             
            'MENDOTA LAKE': {'labels':['Microcystin (nM)',
                                      'OXYGEN DISSOLVED (FIELD METER)',
                                      'Total cyanobacterial cell count (cells/mL)'],
                            'Years':['2018'],
                            'DataBase':'Dataset_US.csv',
                            'InitialP':0.075,
                            'InitialO2': 5 },
                             
            'MONONA LAKE': {'labels':['OXYGEN DISSOLVED (FIELD METER)',
                                          'Total cyanobacterial cell count (cells/mL)',
                                          'TEMPERATURE WATER'],
                            'Years':['2015'],
                            'DataBase':'Combined_Data_for_MO_Merged.csv',
                            'InitialP':0.075,
                            'InitialO2': 5 }}

In [11]:
 
def read_params(lake_name):

        # Initial conditions
        M_0 = 0
        A_0 = 0.05
        B_0 = 0.004  # 0.004
        Q_B_0 = 0.01
        Q_A_0 = 0.01
        
        D_0 = 0.0239  # 0.0239
        Y_0 = 0.025*10
        W_0 = 0.0025*10
        
        v_A_0 = 0
        v_D_0 = 0
        v_Y_0 = 0
        v_W_0 = 0
        
        
        # Data
        LakeYear = {'PIGEON LAKE': ['2021'],
                    'PINE LAKE': ['2017'],
                    'MONONA LAKE': ['2015'],
                    'MENDOTA LAKE': ['2018']}
        
        
        years = LakeYear[lake_name]
        
        yearname = ''
        for year in years:
            yearname = yearname + str(year) + '_'
        
        if lake_name == 'PIGEON LAKE' and years[-1] == '2021':
            coment = "_v6_3Var_"
        else:
            coment = "_v6_3Var_" + yearname
        
        
        # # import parameters
        
        if lake_name in ['MONONA LAKE']:
            rectTemp = 3.0
            B_scale = 0.1
            A_scale = 100
            D_scale = 0.1
            P_0 = 0.01
            P_in = 0.01
            O_0 = 5
        
        elif lake_name in ['MENDOTA LAKE']:
            rectTemp = 3.0
            B_scale = 0.1
            A_scale = 10
            D_scale = 10
            P_0 = 0.075
            P_in = 0.015
            O_0 = 5
        
        elif lake_name in ['PIGEON LAKE']:
            rectTemp = 0.0
            B_scale = 1
            A_scale = 1
            D_scale = 1
            P_0 = 0.05
            P_in = 0.01
            O_0 = 6
        
        elif lake_name in ['PINE LAKE']:
            rectTemp = -2.75
            B_scale = 1.0
            A_scale = 2.0
            D_scale = 0.1
            P_0 = 0.4
            P_in = 0.2
            O_0 = 5
        
        
        
        initial_conditions = [M_0, B_0, A_0,
                              Q_B_0,
                              Q_A_0, P_0,
                              D_0,
                              Y_0, W_0,
                              v_A_0, v_D_0,
                              v_Y_0, v_W_0, O_0]




    
        model_CyB = fullmodel_v1_8.modelCyB()
        model_CyB.initial = initial_conditions
        name_data = './FittedParameters/' + \
            'fitting_parameters_full_variables_v2'+lake_name + coment + '_final' + ".csv"
        
        params_fit = pd.read_csv(name_data)
    
        unknow_params = ["alpha_D", "alpha_Y",
                         "tau_D", "tau_Y",
                         "a_A",
                         "sigma_A",
                         "x_A"]
    
        model_CyB.initial[1] = params_fit["B_0"][0]*B_scale
        # print('B(0)=', model_CyB.initial[1])
        model_CyB.initial[2] = params_fit["A_0"][0]*A_scale
        # print('A(0)=', model_CyB.initial[2])
        model_CyB.initial[6] = params_fit["D_0"][0]*D_scale
        # print('D(0)=', model_CyB.initial[6])
    
        for name in unknow_params:
            model_CyB.params[name] = params_fit[name][0]
    
        # Death Daphnia
    
        # 0.015  # 0.03 Pigeon lake  # 0.09601  # 0.125
        model_CyB.params['p_in'] = P_in
        # model_CyB.params["delta_B-"] = 2.4
        # With Toxine
        model_CyB.toxines = True and (model_CyB.initial[1] > 0)
        # New time scale
        # New time scale
        day_start = pd.to_datetime("2023-05-01").day_of_year
    
        days_end = pd.to_datetime("2023-09-30").day_of_year
    
        model_CyB.t_0 = 0  # May 1, 2023
    
        model_CyB.t_f = days_end - day_start
    
        model_CyB.delta_t = model_CyB.t_f*3
    
        model_CyB.set_linetime()
    
        # Temperature and Zm
        path = './ERA5-Land/' + years[-1]
        TempZmData = pd.read_csv(path+lake_name + 'WaterTemperature.csv')
        TempZmData['Date'] = pd.to_datetime(
            TempZmData['Date'], format='mixed')
    
        tempSamp = TempZmData['lake_mix_layer_temperature']-rectTemp
        Zmsample = (TempZmData['lake_mix_layer_depth_min'] +
                    TempZmData['lake_mix_layer_depth_max'])*0.5
        days = TempZmData['Date'].dt.day_of_year
    
        days = np.array(days) - day_start
    
        model_CyB.get_interpTemp(tempSamp, days)
        model_CyB.get_interpZm(Zmsample, days)
        return model_CyB

In [12]:
def Average(M, model):
    noemptyvalues = ~np.isnan(M)
    return np.trapz(M[noemptyvalues], model.t[noemptyvalues]) / model.t_f
    
# Define solve_model function
def solve_model(model, d_E, z_m):
    import copy
    import numpy as np

    def average(M):
        noemptyvalues = ~np.isnan(M)
        return np.trapz(M[noemptyvalues], model.t[noemptyvalues]) / model.t_f
        
    model_copy = copy.deepcopy(model)
    model_copy.params['d_E'] = d_E
    model_copy.auxZm = z_m
    solution, _ = model_copy.solver()
    M_values, _, _, _, _, _, _, _, _, _, _, _, _, _ = solution.T
    return average(M_values), np.max(M_values[~np.isnan(M_values)])

# Main function
def main(TEMP, lake_name, avar_base, max_base):
    tic = time.time()
    yearname = ''
    for year in InfoLakes[lake_name]['Years']:
        yearname = yearname + str(year) + '_'
    path = "./Vulnerability/"+lake_name + '/'+yearname
    path_max =  path + 'SSP_dE_zm_max_'
    path_mean = path + 'SSP_dE_zm_mean_' 
    
    pool = Pool(NUM_PROCESSES)
    path_max = path_max + f'{TEMP}.csv'
    path_mean = path_mean + f'{TEMP}.csv'
        
    model_CyB_var = read_params(lake_name)

    model_CyB_var.auxTemp = TEMP

    # Iterate over d_values and Z_values in parallel
    results = []
    results_average = []
    results_max = []
    for d_E in d_values:
        async_results = []
        print('Processing...d_E', d_E)
        for z_m in Z_values:
            async_result = pool.apply_async(
                    solve_model, (model_CyB_var, d_E, z_m))  # Provide d_E and z_m here
            async_results.append(async_result)
        results.append([async_result.get()
                                   for async_result in async_results])

    for rows in results: 
        results_average.append([row[0] for row in rows])
        results_max.append([row[1] for row in rows])
    if SAVE_HEAT_SUB:
        Max_data = pd.DataFrame(np.array(results_max) / max_base)
        Max_data.to_csv(path_max)
        Mean_data = pd.DataFrame(np.array(results_average) / avar_base)
        Mean_data.to_csv(path_mean)
        print('Save Data SSP until Temp and d_E: ' + str(TEMP), d_E)


    # if SAVE_HEAT:
    #     Max_data = pd.DataFrame(Max_matrix)
    #     Max_data.to_csv(path_max+ str(TEMP) + '.csv')
    #     Mean_data = pd.DataFrame(Mean_matrix)
    #     Mean_data.to_csv(path_mean+ str(TEMP) + '.csv')

    pool.close()
    pool.join()
    
    toc = time.time()
    print(f'Completed in {toc - tic} seconds')

In [15]:
SSP_temp = [0.5,1.5,3.5]
Lakes = ['MENDOTA LAKE', 'PIGEON LAKE' ,'PINE LAKE', 'MONONA LAKE']
for lake_name in Lakes:
    print('Processing Lake: ', lake_name)
    model_CyB_base = read_params(lake_name)
    solution_base, _ = model_CyB_base.solver()
    M_values_base, _, _, _, _, _, _, _, _, _, _, _, _, _ = solution_base.T
    avar_base = Average(M_values_base, model_CyB_base)
    max_base = np.max(M_values_base[~np.isnan(M_values_base)])
    
    for TEMP in SSP_temp:
        print('Processing Temp: ', TEMP)
        main(TEMP, lake_name, avar_base, max_base)

Processing Lake:  PINE LAKE
Processing Temp:  0.5
Processing...d_E 0.02
Processing...d_E 0.0212
Processing...d_E 0.0224
Processing...d_E 0.0236
Processing...d_E 0.0248
Processing...d_E 0.026
Processing...d_E 0.0272
Processing...d_E 0.0284
Processing...d_E 0.0296
Processing...d_E 0.0308
Processing...d_E 0.032
Processing...d_E 0.0332
Processing...d_E 0.0344
Processing...d_E 0.0356
Processing...d_E 0.0368
Processing...d_E 0.038
Processing...d_E 0.0392
Processing...d_E 0.0404
Processing...d_E 0.0416
Processing...d_E 0.0428
Processing...d_E 0.044
Processing...d_E 0.0452
Processing...d_E 0.0464
Processing...d_E 0.0476
Processing...d_E 0.0488
Processing...d_E 0.05
Processing...d_E 0.0512
Processing...d_E 0.0524
Processing...d_E 0.0536
Processing...d_E 0.0548
Processing...d_E 0.056
Processing...d_E 0.0572
Processing...d_E 0.0584
Processing...d_E 0.0596
Processing...d_E 0.0608
Processing...d_E 0.062
Processing...d_E 0.0632
Processing...d_E 0.0644
Processing...d_E 0.0656
Processing...d_E 0.0668
